# Modeling_ML 9조

- 이 코드는 조용중님 코드를 기초로 하여 catboost 모델을 중점적으로 코딩하였다.

### Library + Data Setting.

In [ ]:
%pip install pip --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
%pip install lightgbm koreanize_matplotlib optuna catboost wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00


In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/DL/ml_competition/ML_competition/data/'

In [ ]:
# path = '../../data/'
house_data = pd.read_csv(path + 'train_final.csv')
test_data = pd.read_csv(path + 'test_final.csv')
# house_data = pd.read_csv(path + 'train_cho_3.csv')
# test_data = pd.read_csv(path + 'test_cho_3.csv')

import re

# 피처 이름에서 특수 문자를 밑줄(_)로 대체하는 함수
def clean_column_names(df):
    df.columns = [re.sub(r'\W+', '_', col) for col in df.columns]
    return df

house_data = clean_column_names(house_data)
test_data = clean_column_names(test_data)

test_data = test_data[house_data.columns]

In [ ]:
house_data.columns, len(house_data.columns)

(Index(['아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', 'k_단지분류_아파트_주상복합등등_',
        'k_세대타입_분양형태_', 'k_관리방식', 'k_복도유형', 'k_난방방식', 'k_전체동수', 'k_전체세대수',
        'k_건설사_시공사_', 'k_시행사', 'k_연면적', 'k_주거전용면적', 'k_전용면적별세대현황_60_이하_',
        'k_전용면적별세대현황_60_85_이하_', 'k_85_135_이하', 'k_135_초과', '건축면적', '주차대수',
        '기타_의무_임대_임의_1_2_3_4', '좌표X', '좌표Y', 'target', 'distance_score', '구',
        '동', '계약연도', '계약월', '급지', 'address', '신축여부', '이자율'],
       dtype='object'),
 37)

## lightGBM

In [ ]:
col = ['아파트명', '전용면적', '계약년월', '층', '건축년도', '도로명', 'k_단지분류_아파트_주상복합등등_',\
        'k_세대타입_분양형태_', 'k_관리방식', 'k_복도유형', 'k_난방방식', 'k_전체동수', 'k_건설사_시공사_',\
        'k_시행사', '좌표X', '좌표Y', 'target', '구', '동', '이자율', 'distance_score',\
        '급지']

# col_down = ['아파트명', '전용면적', '층', '건축년도',\
#        '좌표X', '좌표Y', 'target', 'year', 'month',\
#        '구', '동', 'distance_score', '급지','이자율','신축여부']


house_data = house_data[col]
test_data = test_data[col]

In [ ]:
# house_data.drop(['계약년월','address','date','stations_count','Unnamed: 0'],axis=1,inplace=True)
# test_data.drop(['계약년월','address','date','stations_count','Unnamed: 0'],axis=1,inplace=True)

### 범주형/연속형/전용면적 로그 처리등

In [ ]:
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in house_data.columns:
    if pd.api.types.is_numeric_dtype(house_data[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

연속형 변수: ['전용면적', '계약년월', '층', '건축년도', 'k_연면적', 'k_관리비부과면적', 'k_전용면적별세대현황_60_이하_', 'k_전용면적별세대현황_60_85_이하_', 'k_85_135_이하', '좌표X', '좌표Y', 'target', 'year', 'month', '신축여부', '이자율', 'distance_score', 'is_top20']
범주형 변수: ['도로명', 'k_단지분류_아파트_주상복합등등_', 'k_세대타입_분양형태_', 'k_관리방식', 'k_복도유형', 'k_난방방식', 'k_건설사_시공사_', 'k_시행사', '청소비관리형태', '기타_의무_임대_임의_1_2_3_4', '구', '동', '급지']


In [ ]:
# train과 test 데이터를 합쳐서 하나의 데이터프레임으로 만듭니다.
combined_data = pd.concat([house_data, test_data], axis=0)

# 전용면적 로그 변환
#combined_data['전용면적'] = np.log1p(combined_data['전용면적'])

# 각 범주형 변수에 대해 LabelEncoder를 적용합니다.
label_encoders = {}

for col in categorical_columns_v2:
    lbl = LabelEncoder()
    combined_data[col] = combined_data[col].astype(str)  # 데이터 타입을 문자열로 변환
    combined_data[col] = lbl.fit_transform(combined_data[col])
    label_encoders[col] = lbl  # 나중에 후처리를 위해 레이블인코더를 저장해둡니다.

# 인코딩된 데이터를 다시 train과 test 데이터로 나눕니다.
train_data_encoded = combined_data.iloc[:len(house_data)]
test_data_encoded = combined_data.iloc[len(house_data):]

In [ ]:
test_data_encoded.shape,train_data_encoded.shape

((9272, 31), (1118822, 31))

In [ ]:
# # 데이터 타입 최적화
# for df in [train_data_encoded, test_data_encoded]:
#     df['아파트명'] = df['아파트명'].astype('int16')
#     df['전용면적'] = df['전용면적'].astype('float32')
#     df['층'] = df['층'].astype('int8')
#     df['건축년도'] = df['건축년도'].astype('int16')
#     df['도로명'] = df['도로명'].astype('int32')
#     df['k-복도유형'] = df['k-복도유형'].astype('int8')
#     df['k-난방방식'] = df['k-난방방식'].astype('int8')
#     df['k-전체세대수'] = df['k-전체세대수'].astype('float32')
#     df['주차대수'] = df['주차대수'].astype('float32')
#     df['기타/의무/임대/임의=1/2/3/4'] = df['기타/의무/임대/임의=1/2/3/4'].astype('int8')
#     df['좌표X'] = df['좌표X'].astype('float32')
#     df['좌표Y'] = df['좌표Y'].astype('float32')
#     df['target'] = df['target'].astype('float32')
#     df['계약연도'] = df['계약연도'].astype('int16')
#     df['계약월'] = df['계약월'].astype('int8')
#     df['구'] = df['구'].astype('int8')
#     df['동'] = df['동'].astype('int16')
#     df['distance_score'] = df['distance_score'].astype('int8')
#     df['급지'] = df['급지'].astype('int8')

In [ ]:
# Target과 독립변수들을 분리해줍니다.
y_train = train_data_encoded['target']
X_train = train_data_encoded.drop(['target'], axis=1)
X_test = test_data_encoded.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#### 2023년 데이트를 검증데이터로 처리

In [ ]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.cluster import KMeans

year = 2022

# 2023년 데이터를 선택
train_2023 = train_data_encoded[(train_data_encoded['year'] >year)]

# 테스트 데이터 (2023년 7월부터 9월까지의 데이터)
test_data = test_data_encoded.drop(['target'], axis=1)
# 나머지 훈련 데이터
train_data = train_data_encoded[train_data_encoded['year'] <= year]

# 데이터 표준화
scaler = StandardScaler()
train_2023_scaled = scaler.fit_transform(train_2023.drop('target',axis=1))
test_data_scaled = scaler.transform(test_data)

# K-Means 클러스터링
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(train_2023_scaled)


# 각 데이터 포인트의 클러스터 레이블
train_2023['cluster'] = kmeans.labels_
test_labels = kmeans.predict(test_data_scaled)

# 비율을 맞추기 위한 검증 데이터 선택
val_ratio = 1 / 2.5  # 5:1 비율에서 검증 데이터의 비율
val_indices = []

for cluster in train_2023['cluster'].unique():
    cluster_indices = train_2023[train_2023['cluster'] == cluster].index
    n_val_samples = int(len(cluster_indices) * val_ratio)
    val_indices.extend(cluster_indices[:n_val_samples])

# 검증 데이터 선택
val_indices = np.array(val_indices)
X_val_2023 = train_2023.loc[val_indices]
train_2023_remaining = train_2023.drop(val_indices)

# 최종 훈련 데이터와 검증 데이터 결합
final_train_data = pd.concat([train_data, train_2023_remaining])
final_val_data = X_val_2023

# 특징과 타겟 변수 분리
X_train = final_train_data.drop(['target', 'cluster'], axis=1)
y_train = final_train_data['target']
X_val = final_val_data.drop(['target', 'cluster'], axis=1)
y_val = final_val_data['target']
X_test = test_data

# 결과 출력
print("최종 훈련 데이터 크기:", X_train.shape)
print("최종 검증 데이터 크기:", X_val.shape)

최종 훈련 데이터 크기: (1111804, 30)
최종 검증 데이터 크기: (7018, 30)


In [ ]:
len(train_2023)

17548

### Simple lightGBM

In [ ]:
import wandb
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Start a new wandb run to track this script
wandb.init(
    project="re_price_prediction",
    name="in_아파트명_xval_2023+LGBM",
    config={
        "learning_rate": 0.01,
        "n_estimators": 100000,
        "boosting_type": 'gbdt',
        "objective": 'regression',
        "metric": 'rmse',
        "dataset": "train",
    }
)

# Define the LightGBM regressor
gbm = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='regression',
    learning_rate=0.01,
    n_estimators=100000,
    metric='rmse',
)

# Callback function to log evaluation metrics to wandb
def log_to_wandb(env):
    for data_name, eval_name, eval_result, stdv in env.evaluation_result_list:
        metric_name = f"{data_name}_{eval_name}"
        wandb.log({metric_name: eval_result})

# Fit the model
gbm.fit(
    X_train, y_train,
    eval_set=[[X_train, y_train], [X_val, y_val]],
    eval_metric='rmse',
    categorical_feature="auto",
    callbacks=[
        lgb.early_stopping(stopping_rounds=1000),
        lgb.log_evaluation(period=10, show_stdv=True),
        log_to_wandb
    ]
)

# [optional] finish the wandb run
wandb.finish()


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021805 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2345
[LightGBM] [Info] Number of data points in the train set: 895057, number of used features: 19
[LightGBM] [Info] Start training from score 58003.281782
Training until validation scores don't improve for 1000 rounds
[10]	training's rmse: 43098.4	valid_1's rmse: 43376.2
[20]	training's rmse: 40193.8	valid_1's rmse: 40470.2
[30]	training's rmse: 37604.4	valid_1's rmse: 37879.3
[40]	training's rmse: 35258.5	valid_1's rmse: 35534.6
[50]	training's rmse: 33170	valid_1's rmse: 33446.9
[60]	training's rmse: 31301.9	valid_1's rmse: 31578.7
[70]	training's rmse: 29633.3	valid_1's rmse: 29906
[80]	training's rmse: 28145.4	valid_1's rmse: 28413.1
[90]	training's rmse: 26807.1	valid_1's rmse: 27073.5
[100]	training's rmse: 25610.9	valid_1's rms

training_rmse,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_1_rmse,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_rmse,3491.65594
valid_1_rmse,5662.86985


In [ ]:
%%time
X_test = test_data_encoded.drop(['target'], axis=1)
# 최적의 반복 횟수로 X_test에 대한 예측 수행
real_test_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print(real_test_pred)
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)

[182276.30017586 279194.92271447 323276.49978819 ...  82530.05045453
  70650.25134222  71106.86989016]
CPU times: user 1min 34s, sys: 322 ms, total: 1min 35s
Wall time: 13.2 s


In [ ]:
### 용중님의 베스트 output과 비교.
y_best = pd.read_csv('output_cho.csv',index_col=0)

rmse = np.sqrt(mean_squared_error(y_best, real_test_pred))
rmse

(3806.4963307292837,
 array([182276.30017586, 279194.92271447, 323276.49978819, ...,
         82530.05045453,  70650.25134222,  71106.86989016]))

In [ ]:
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
big_error_20_idx = abs(y_best-preds_df).sort_values('target',ascending=False)[:20]
big_error_20_idx

,target
5495,88149
5496,75465
5024,65437
5490,62231
6056,55505
8593,51663
264,46006
6059,43304
5554,40204
8568,39718


### lightGBM + parmameter tunning

In [ ]:
# 파라미터 정의
wandb.init(
    project="re_price_prediction",
    group="features",
    reinit=True,
    config={
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': 53149,
        'learning_rate': 0.010204448913134991,
        'num_leaves': 98,
        'max_depth': 10,
        'min_child_samples': 8,
        'subsample': 0.5668227534307944,
        'colsample_bytree': 0.6033862727269486,
        'reg_alpha': 2.59062446980025e-06,
        'reg_lambda': 2.979573198070953e-05,
        'force_row_wise':True,  # 행 기반 멀티 스레딩 강제 적용
        #'force_col_wise':True,  # 열 기반 멀티 스레딩 강제 적용 (메모리 최적화 시)
        # 'device': 'cuda',  # GPU 사용 설정
        # 'gpu_platform_id': 0,  # GPU 플랫폼 ID (일반적으로 0)
        # 'gpu_device_id': 0    # GPU 디바이스 ID (여러 GPU가 있는 경우 선택)}
    }
)

params = wandb.config

# LGBMRegressor 모델 생성
gbm = lgb.LGBMRegressor(**params)

# Callback function to log evaluation metrics to wandb
def log_to_wandb(env):
    for data_name, eval_name, eval_result, stdv in env.evaluation_result_list:
        metric_name = f"{data_name}_{eval_name}"
        wandb.log({metric_name: eval_result})

# 모델 학습
gbm.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_metric='rmse',
        categorical_feature="auto",
        callbacks=[lgb.early_stopping(stopping_rounds=50),
                   lgb.log_evaluation(period=10, show_stdv=True),
                   log_to_wandb])

# [optional] finish the wandb run
wandb.finish()

[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 895057, number of used features: 30
[LightGBM] [Info] Start training from score 58003.281782
Training until validation scores don't improve for 50 rounds
[10]	training's rmse: 43045.6	valid_1's rmse: 43315.7
[20]	training's rmse: 40075.4	valid_1's rmse: 40342.5
[30]	training's rmse: 37372.1	valid_1's rmse: 37637.9
[40]	training's rmse: 34831.6	valid_1's rmse: 35103.6
[50]	training's rmse: 32575.8	valid_1's rmse: 32847.4
[60]	training's rmse: 30574.3	valid_1's rmse: 30844.5
[70]	training's rmse: 28608.3	valid_1's rmse: 28881.1
[80]	training's rmse: 26986.4	valid_1's rmse: 27264.6
[90]	training's rmse: 25431.9	valid_1's rmse: 25709
[100]	training's rmse: 23934.4	valid_1's rmse: 24213.2
[110]	training's rmse: 22672.9	valid_1's rmse: 22956.9
[120]	training's rmse: 21513.5	valid_1's rmse: 21798.7
[130]	training's rmse: 20436.8	valid_1's rmse: 20719.7
[140]	training's rmse: 19553.1	valid_1's rmse: 198

LGBMRegressor(colsample_bytree=0.6033862727269486, force_row_wise=True,
              learning_rate=0.010204448913134991, max_depth=10, metric='rmse',
              min_child_samples=8, n_estimators=53149, num_leaves=98,
              objective='regression', reg_alpha=2.59062446980025e-06,
              reg_lambda=2.979573198070953e-05, subsample=0.5668227534307944)

In [ ]:
%%time
X_test = test_data_encoded.drop(['target'], axis=1)
# 최적의 반복 횟수로 X_test에 대한 예측 수행
real_test_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print(real_test_pred)
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=True)

[191029.35805258 277404.48445954 326778.59061862 ...  84713.07862366
  70173.0417146   68044.73933144]
CPU times: user 1min 47s, sys: 7.85 ms, total: 1min 47s
Wall time: 11.7 s


### lightGBM + optuna : 하이퍼파라미터 최적화

In [ ]:
!pip install optuna

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/15/da/68883911855d8b4d521f9a370e4e6aab8232b91c1d8d5a8348c4680c6642/optuna-3.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/df/ed/c884465c33c25451e4a5cd4acad154c29e5341e3214e220e7f3478aa4b0d/alembic-1.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for colorlog from https://files.pythonhosted.org/packages/f3/18/3e867ab37a24fdf073c1617b9c7830e06ec270b1ea4694a624038fc40a03/colorlog-6.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy>=1.3.0 from https://files.pythonhosted.org/packages/bf/90/0ce8c76cd2ef8a5e0838d61d6648a1193859e1990bc1d65c8b0fd1f77ca8/SQLAlchemy-2.0.31-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for Mako from https://files.pythonhosted.org/packages/03/62/70f5a0c2dd208f9f3f2f9afd103aec42ee4d9ad2401d7

In [ ]:
import optuna

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000),
        'max_depth': trial.suggest_int('max_depth', 9, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.05, 0.15),
        'n_estimators': trial.suggest_int('n_estimators', 10000, 100000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-8, 1.0),
        'force_col_wise':True,  # 행 기반 멀티 스레딩 강제 적용

        # 'device': 'cuda',
        # 'gpu_platform_id': 0,
        # 'gpu_device_id': 0
    }

    gbm = lgb.LGBMRegressor(**params)

    gbm.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_metric='rmse',
            categorical_feature="auto",
            callbacks=[ lgb.early_stopping(stopping_rounds=50),
                        lgb.log_evaluation(period=10, show_stdv=True)])

    preds = gbm.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    return rmse


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=-1, timeout=3600)  # 1시간 동안 실행

print("Best parameters:", study.best_params)
print("Best RMSE:", study.best_value)

# 최적의 파라미터로 최종 모델 학습
best_params = study.best_params
best_params['metric'] = 'rmse'  # metric을 다시 추가

# best_params['device'] = 'cuda'
# best_params['gpu_platform_id'] = 0
# best_params['gpu_device_id'] = 0
best_model = lgb.LGBMRegressor(**best_params)
best_model.fit(X_train, y_train,
               eval_set=[(X_train, y_train), (X_val, y_val)],
               eval_metric='rmse',
               categorical_feature="auto",
               callbacks=[lgb.early_stopping(stopping_rounds=50),
                          lgb.log_evaluation(period=10, show_stdv=True)])

[I 2024-07-15 09:21:08,260] A new study created in memory with name: no-name-f9460603-1b23-400d-a3e4-c0ad7ea4e37e


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.236235 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.207182 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.218140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 4.031575 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1835
[LightGBM] [Info] Number

## catBoost

### Simple catBoost

In [ ]:
import wandb
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import optuna
import wandb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# 데이터 분할
X = house_data.drop('target', axis=1)
y = house_data['target']
X_test = test_data.drop('target', axis=1)

categorical_features = X.select_dtypes(include=['object']).columns.tolist()

for col in categorical_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost Pool 생성
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)

results=[]
# wandb 초기화
wandb.init(
    project="cat-CatBoostRegressor",
    group='test_nonlog',
    reinit=True,
    config={
        'iterations': 100000,#trial.suggest_int('iterations', 1500, 3000),
        # 'learning_rate':0.009,
        # 'depth':10,
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'random_seed': 42,
        'early_stopping_rounds': 50,
        'bootstrap_type':'Bernoulli',
        'task_type': 'GPU',  # GPU 사용 설정
        'devices': '0:1:2:3:4:5:6:7:8:9:10:11:12:13:14:15'    # 여러 GPU 사용 설정 (예: 0번과 1번 GPU 사용)
    }
)

params = wandb.config

# 모델 설정
model = CatBoostRegressor(**params)

# 모델 학습
model.fit(train_pool, eval_set=val_pool, verbose=10)



# # 데이터 분할
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2024)

# # CatBoost Pool 생성
# train_pool = Pool(X_train, y_train, cat_features=categorical_features)
# val_pool = Pool(X_val, y_val, cat_features=categorical_features)

# model.fit(train_pool, eval_set=val_pool, verbose=10)



# 모델 평가
y_pred = model.predict(X_val)
vrmse = np.sqrt(mean_squared_error(y_pred, y_val))
print("Validation RMSE:", vrmse)

# 테스트 데이터 예측 (예제로 X_val 사용)
y_test_pred = model.predict(X_test)
print("Test Predictions:", y_test_pred)

# 사용된 하이퍼파라미터 확인 및 저장
params_used = model.get_params()
print("Model parameters:\n", params_used)

results.append(params_used)

# wandb config에 하이퍼파라미터 저장
wandb.config.update(params_used)

# wandb 종료
wandb.finish()

#
preds_df = pd.DataFrame(y_test_pred.astype(int), columns=["target"])

### 용중님의 베스트 output과 비교.
y_best = pd.read_csv(path + 'output_cho.csv',index_col=0)
rmse = np.sqrt(mean_squared_error(y_best, preds_df))

filename = path + f'output_{vrmse}_{rmse}.csv'
preds_df.to_csv(filename, index=False)
rmse

Learning rate set to 0.034827
0:	learn: 45238.8118400	test: 45510.4334783	best: 45510.4334783 (0)	total: 34ms	remaining: 56m 40s
10:	learn: 36030.0411681	test: 36264.9788543	best: 36264.9788543 (10)	total: 332ms	remaining: 50m 21s
20:	learn: 29648.7742429	test: 29855.2677688	best: 29855.2677688 (20)	total: 618ms	remaining: 49m 4s
30:	learn: 25243.9701612	test: 25398.6074386	best: 25398.6074386 (30)	total: 904ms	remaining: 48m 34s
40:	learn: 22093.3494668	test: 22234.9867544	best: 22234.9867544 (40)	total: 1.18s	remaining: 48m 4s
50:	learn: 19939.5680760	test: 20084.2207779	best: 20084.2207779 (50)	total: 1.47s	remaining: 47m 55s
60:	learn: 18385.5748939	test: 18542.2659503	best: 18542.2659503 (60)	total: 1.75s	remaining: 47m 48s
70:	learn: 17209.0672272	test: 17395.7840453	best: 17395.7840453 (70)	total: 2.04s	remaining: 47m 45s
80:	learn: 16344.7649206	test: 16528.2733785	best: 16528.2733785 (80)	total: 2.32s	remaining: 47m 40s
90:	learn: 15682.9210426	test: 15867.5735414	best: 15867.

8397.380719268871

In [ ]:
X_train.columns

Index(['아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', 'k_단지분류_아파트_주상복합등등_',
       'k_세대타입_분양형태_', 'k_관리방식', 'k_복도유형', 'k_난방방식', 'k_전체동수', 'k_전체세대수',
       'k_건설사_시공사_', 'k_시행사', 'k_연면적', 'k_주거전용면적', 'k_전용면적별세대현황_60_이하_',
       'k_전용면적별세대현황_60_85_이하_', 'k_85_135_이하', 'k_135_초과', '건축면적', '주차대수',
       '기타_의무_임대_임의_1_2_3_4', '좌표X', '좌표Y', 'distance_score', '구', '동', '계약연도',
       '계약월', '급지', 'address', '신축여부', '이자율'],
      dtype='object')

In [ ]:
X_test.columns

Index(['아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', 'k_단지분류_아파트_주상복합등등_',
       'k_세대타입_분양형태_', 'k_관리방식', 'k_복도유형', 'k_난방방식', 'k_전체동수', 'k_전체세대수',
       'k_건설사_시공사_', 'k_시행사', 'k_연면적', 'k_주거전용면적', 'k_전용면적별세대현황_60_이하_',
       'k_전용면적별세대현황_60_85_이하_', 'k_85_135_이하', 'k_135_초과', '건축면적', '주차대수',
       '기타_의무_임대_임의_1_2_3_4', '좌표X', '좌표Y', '구', '동', 'distance_score', '계약연도',
       '계약월', '급지', 'address', '신축여부', '이자율'],
      dtype='object')

In [ ]:
X.iloc[1,11],X_test.iloc[1,11]

('개별난방', '지역난방')

In [ ]:
categorical_features[10]

'구'

In [ ]:
model.get_all_params()

{'nan_mode': 'Min',
 'gpu_ram_part': 0.95,
 'eval_metric': 'RMSE',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'FeatureFreq:CtrBorderCount=15:CtrBorderType=Median:Prior=0/1'],
 'iterations': 100000,
 'fold_permutation_block': 64,
 'leaf_estimation_method': 'Newton',
 'observations_to_bootstrap': 'TestOnly',
 'od_pval': 0,
 'random_score_type': 'NormalWithModelSizeDecrease',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'ctr_history_unit': 'Sample',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'devices': '0:1:2:3:4:5:6:7:8:9:10:11:12:13:14:15',
 'eval_fraction': 0,
 'pinned_memory_bytes': '104857600',
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'od_type': 'Iter',
 'rsm': 1,
 'boost_from_average': 

In [ ]:
%%time
X_test = test_data.drop('target',axis=1)# 최적의 반복 횟수로 X_test에 대한 예측 수행
real_test_pred = model.predict(X_test)
real_test_pred = np.expm1(real_test_pred)
print(real_test_pred)
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)

[189779.95566699 300198.75087941 308481.22040741 ...  80494.54977493
  73554.43524595  72974.17087784]
CPU times: user 198 ms, sys: 106 ms, total: 304 ms
Wall time: 114 ms


In [ ]:
y_best = pd.read_csv('output_cho.csv',index_col=0)
y_best.rename(columns={'target':'LGBM'})
y_best.columns

Index(['target'], dtype='object')

In [ ]:
### 용중님의 베스트 output과 비교.
y_best = pd.read_csv('output_cho.csv',index_col=0)
y_best = y_best.rename(columns={'target':'LGBM'})
y_best['CAT'] = preds_df['target']

rmse = np.sqrt(mean_squared_error(y_best['LGBM'], y_best['CAT']))
print(rmse)

8943.523700059905


In [ ]:
w = [0.6,0.4]
submission = pd.DataFrame(columns=["target"])
submission['target'] = y_best @ w
submission.to_csv('submission.csv',index=False)

rmse = np.sqrt(mean_squared_error(y_best['LGBM'], submission['target']))
rmse

3577.4094800239623

### K-fold + Cat

In [ ]:
import wandb
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import optuna
import wandb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_predict


# 데이터 분할
X = house_data.drop('target', axis=1)
y = house_data['target']
X_test = test_data.drop('target', axis=1)

categorical_features = X.select_dtypes(include=['object']).columns.tolist()

for col in categorical_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# 데이터 분할
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# CatBoost Pool 생성
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)

results=[]
# wandb 초기화
wandb.init(
    project="cat-CatBoostRegressor",
    group='test_log',
    reinit=True,
    config={
        'iterations': 100000,#trial.suggest_int('iterations', 1500, 3000),
        'learning_rate': 0.032,
        'depth': 10,
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'random_seed': 42,
        'early_stopping_rounds': 50,
        'bootstrap_type':'Bernoulli',
        'task_type': 'GPU',  # GPU 사용 설정
        'devices': '0:1:2:3:4:5:6:7:8:9:10:11:12:13:14:15'    # 여러 GPU 사용 설정 (예: 0번과 1번 GPU 사용)
    }
)

params = wandb.config

# 모델 설정
model = CatBoostRegressor(**params)

kf = KFold(n_splits=2,shuffle=True,random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):

    X_train = X.iloc[train_idx].reset_index(drop = True)
    y_train = y.iloc[train_idx].reset_index(drop = True)

    X_val = X.iloc[val_idx].reset_index(drop = True)
    y_val = y.iloc[val_idx].reset_index(drop = True)

    # CatBoost Pool 생성
    train_pool = Pool(X_train, y_train, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    # 모델 학습
    model.fit(train_pool, eval_set=val_pool, verbose=10)

    # 모델 평가
    y_pred = model.predict(X_val)
    vrmse = np.sqrt(mean_squared_error(y_pred, y_val))
    print("Validation RMSE:", vrmse)

# 테스트 데이터 예측 (예제로 X_val 사용)
y_test_pred = model.predict(X_test)
print("Test Predictions:", y_test_pred)

# 사용된 하이퍼파라미터 확인 및 저장
params_used = model.get_params()
print("Model parameters:\n", params_used)

results.append(params_used)

# wandb config에 하이퍼파라미터 저장
wandb.config.update(params_used)

# wandb 종료
wandb.finish()

#
preds_df = pd.DataFrame(y_test_pred.astype(int), columns=["target"])

### 용중님의 베스트 output과 비교.
y_best = pd.read_csv(path + 'output_cho.csv',index_col=0)
rmse = np.sqrt(mean_squared_error(y_best, preds_df))

filename = path + f'output_{vrmse}_{rmse}.csv'
preds_df.to_csv(filename, index=False)
print(rmse)


0:	learn: 45062.7027804	test: 45504.3895515	best: 45504.3895515 (0)	total: 55.3ms	remaining: 1h 32m 9s
10:	learn: 35344.6528856	test: 35690.5186098	best: 35690.5186098 (10)	total: 539ms	remaining: 1h 21m 42s
20:	learn: 28413.8819147	test: 28675.1697221	best: 28675.1697221 (20)	total: 1s	remaining: 1h 19m 48s
30:	learn: 23491.5356122	test: 23723.8913614	best: 23723.8913614 (30)	total: 1.46s	remaining: 1h 18m 36s
40:	learn: 19992.7277126	test: 20193.0858744	best: 20193.0858744 (40)	total: 1.92s	remaining: 1h 18m 7s
50:	learn: 17514.7484147	test: 17704.2632581	best: 17704.2632581 (50)	total: 2.4s	remaining: 1h 18m 21s
60:	learn: 15729.6035157	test: 15933.8313233	best: 15933.8313233 (60)	total: 2.87s	remaining: 1h 18m 25s
70:	learn: 14427.5071027	test: 14635.6075256	best: 14635.6075256 (70)	total: 3.34s	remaining: 1h 18m 21s
80:	learn: 13452.1126709	test: 13673.5311656	best: 13673.5311656 (80)	total: 3.83s	remaining: 1h 18m 46s
90:	learn: 12734.6339706	test: 12970.8473708	best: 12970.84737

10555.795696555731


In [ ]:
0.026764	10	9.853596	0.139528	0.306367

In [ ]:
import wandb
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import optuna
import wandb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# 데이터 분할
X = house_data.drop('target', axis=1)
y = house_data['target']
X_test = test_data.drop('target', axis=1)

categorical_features = X.select_dtypes(include=['object']).columns.tolist()

for col in categorical_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost Pool 생성
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)

results=[]
# wandb 초기화
wandb.init(
    project="cat-CatBoostRegressor",
    group='test_nonlog',
    reinit=True,
    config={
        'iterations': 100000,#trial.suggest_int('iterations', 1500, 3000),
        'learning_rate': 0.026764,
        'depth': 10,
        'l2_leaf_reg': 9.853596,
        'subsample': 0.139528,
        'random_strength': 306367,
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'random_seed': 2024,
        'early_stopping_rounds': 50,
        'bootstrap_type':'Bernoulli',
        'task_type': 'GPU',  # GPU 사용 설정
        'devices': '0:1:2:3:4:5:6:7:8:9:10:11:12:13:14:15'    # 여러 GPU 사용 설정 (예: 0번과 1번 GPU 사용)
    }
)

params = wandb.config

# 모델 설정
model = CatBoostRegressor(**params)

# 모델 학습
model.fit(train_pool, eval_set=val_pool, verbose=10)



# # 데이터 분할
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2024)

# # CatBoost Pool 생성
# train_pool = Pool(X_train, y_train, cat_features=categorical_features)
# val_pool = Pool(X_val, y_val, cat_features=categorical_features)

# model.fit(train_pool, eval_set=val_pool, verbose=10)



# 모델 평가
y_pred = model.predict(X_val)
vrmse = np.sqrt(mean_squared_error(y_pred, y_val))
print("Validation RMSE:", vrmse)

# 테스트 데이터 예측 (예제로 X_val 사용)
y_test_pred = model.predict(X_test)
print("Test Predictions:", y_test_pred)

# 사용된 하이퍼파라미터 확인 및 저장
params_used = model.get_params()
print("Model parameters:\n", params_used)

results.append(params_used)

# wandb config에 하이퍼파라미터 저장
wandb.config.update(params_used)

# wandb 종료
wandb.finish()

#
preds_df = pd.DataFrame(y_test_pred.astype(int), columns=["target"])

### 용중님의 베스트 output과 비교.
y_best = pd.read_csv(path + 'output_cho.csv',index_col=0)
rmse = np.sqrt(mean_squared_error(y_best, preds_df))

filename = path + f'output_{vrmse}_{rmse}.csv'
preds_df.to_csv(filename, index=False)
rmse

0:	learn: 45725.9862461	test: 46007.0334866	best: 46007.0334866 (0)	total: 56.1ms	remaining: 1h 33m 34s
10:	learn: 40588.7740941	test: 40825.5714076	best: 40825.5714076 (10)	total: 567ms	remaining: 1h 25m 49s
20:	learn: 37042.4886461	test: 37226.4182280	best: 37226.4182280 (20)	total: 1.04s	remaining: 1h 22m 29s
30:	learn: 34454.2653017	test: 34603.6547216	best: 34603.6547216 (30)	total: 1.5s	remaining: 1h 20m 39s
40:	learn: 32166.0222560	test: 32274.9858195	best: 32274.9858195 (40)	total: 2.01s	remaining: 1h 21m 42s
50:	learn: 30095.5577380	test: 30184.9826274	best: 30184.9826274 (50)	total: 2.52s	remaining: 1h 22m 20s
60:	learn: 28710.1010149	test: 28766.1771117	best: 28766.1771117 (60)	total: 3.02s	remaining: 1h 22m 21s
70:	learn: 27285.7762026	test: 27335.8010963	best: 27335.8010963 (70)	total: 3.49s	remaining: 1h 21m 49s
80:	learn: 26656.6591974	test: 26690.5477885	best: 26690.5477885 (80)	total: 3.97s	remaining: 1h 21m 39s
90:	learn: 25738.0674876	test: 25741.0319342	best: 25741.

7591.544749557503

### catBoost + optuna

In [ ]:
import optuna
import wandb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# 데이터 분할
X = house_data.drop('target', axis=1)
y = house_data['target']
X_test = test_data.drop('target', axis=1)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# CatBoost Pool 생성
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)

for col in categorical_features:
    X_train[col] = X_train[col].astype(str)
    X_val[col] = X_val[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# 결과 저장 리스트
results = []
wandb.init(project="cat-CatBoostRegressor",group='feature_reduce')

def objective(trial):
    # 하이퍼파라미터 정의
    params = {
        'iterations': 500,#trial.suggest_int('iterations', 1500, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.03),
        'depth': trial.suggest_int('depth', 7, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'random_strength': trial.suggest_float('random_strength', 0, 2),
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'random_seed': 42,
        'early_stopping_rounds': 50,
        'bootstrap_type':'Bernoulli',
        'task_type': 'GPU',  # GPU 사용 설정
        'devices': '0:1:2:3:4:5:6:7:8:9:10'    # 여러 GPU 사용 설정 (예: 0번과 1번 GPU 사용)
    }

    print(f"Trial {trial.number} params: {params}")

    # 모델 생성
    model = CatBoostRegressor(**params)

    # 모델 학습
    model.fit(
        train_pool,
        eval_set=val_pool,
        verbose=10
    )

    # 예측
    preds = model.predict(X_val)
    # RMSE 계산
    rmse = np.sqrt(np.mean((y_val - preds) ** 2))

    # 결과 저장
    result = params.copy()
    result['rmse'] = rmse
    results.append(result)
    wandb.log(result)

    return rmse


# 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

wandb.finish()

# 최적의 하이퍼파라미터 출력
print('Best trial:')
trial = study.best_trial
print('RMSE: {}'.format(trial.value))
print('Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

log_data = pd.read_csv(path + 'log_catboost_optuna_colab.csv',index_col=0)
final_results = pd.concat([log_data, pd.DataFrame(results)],axis=0).sort_values('rmse')
final_results.to_csv(path + 'log_catboost_optuna_colab.csv')


[I 2024-07-18 22:43:43,634] A new study created in memory with name: no-name-3ac14969-50b6-4fdb-b8a0-a22b6486151c


Trial 0 params: {'iterations': 500, 'learning_rate': 0.0032123249798414097, 'depth': 7, 'l2_leaf_reg': 4.935999191256432, 'subsample': 0.8977077197011215, 'random_strength': 1.6420947216575177, 'loss_function': 'RMSE', 'eval_metric': 'RMSE', 'random_seed': 42, 'early_stopping_rounds': 50, 'bootstrap_type': 'Bernoulli', 'task_type': 'GPU', 'devices': '0:1:2:3:4:5:6:7:8:9:10'}
0:	learn: 46262.2558294	test: 46539.7762065	best: 46539.7762065 (0)	total: 41.9ms	remaining: 20.9s
10:	learn: 45188.9341377	test: 45459.5906012	best: 45459.5906012 (10)	total: 408ms	remaining: 18.1s
20:	learn: 44154.8312068	test: 44418.3886905	best: 44418.3886905 (20)	total: 749ms	remaining: 17.1s
30:	learn: 43167.4228248	test: 43428.2177991	best: 43428.2177991 (30)	total: 1.08s	remaining: 16.3s
40:	learn: 42214.1356522	test: 42472.7234844	best: 42472.7234844 (40)	total: 1.41s	remaining: 15.8s
50:	learn: 41287.3005593	test: 41541.8281904	best: 41541.8281904 (50)	total: 1.75s	remaining: 15.4s
60:	learn: 40397.304968

[I 2024-07-18 22:44:03,926] Trial 0 finished with value: 20088.34475058694 and parameters: {'learning_rate': 0.0032123249798414097, 'depth': 7, 'l2_leaf_reg': 4.935999191256432, 'subsample': 0.8977077197011215, 'random_strength': 1.6420947216575177}. Best is trial 0 with value: 20088.34475058694.


Trial 1 params: {'iterations': 500, 'learning_rate': 0.01765636819383803, 'depth': 8, 'l2_leaf_reg': 8.241537202724292, 'subsample': 0.5759460512294547, 'random_strength': 1.0091541507715798, 'loss_function': 'RMSE', 'eval_metric': 'RMSE', 'random_seed': 42, 'early_stopping_rounds': 50, 'bootstrap_type': 'Bernoulli', 'task_type': 'GPU', 'devices': '0:1:2:3:4:5:6:7:8:9:10'}
0:	learn: 45749.4542618	test: 46025.2367195	best: 46025.2367195 (0)	total: 49ms	remaining: 24.4s
10:	learn: 40202.4268666	test: 40450.9733248	best: 40450.9733248 (10)	total: 471ms	remaining: 20.9s
20:	learn: 35621.4769523	test: 35824.2963660	best: 35824.2963660 (20)	total: 852ms	remaining: 19.4s
30:	learn: 31800.9644527	test: 31972.5540039	best: 31972.5540039 (30)	total: 1.25s	remaining: 19s
40:	learn: 28572.7141500	test: 28726.7134963	best: 28726.7134963 (40)	total: 1.65s	remaining: 18.5s
50:	learn: 25905.0746918	test: 26052.3533828	best: 26052.3533828 (50)	total: 2.04s	remaining: 17.9s
60:	learn: 23676.2012633	test

[I 2024-07-18 22:44:28,467] Trial 1 finished with value: 10562.796247064338 and parameters: {'learning_rate': 0.01765636819383803, 'depth': 8, 'l2_leaf_reg': 8.241537202724292, 'subsample': 0.5759460512294547, 'random_strength': 1.0091541507715798}. Best is trial 1 with value: 10562.796247064338.


...

[I 2024-07-18 22:47:38,762] Trial 8 finished with value: 9044.410134286629 and parameters: {'learning_rate': 0.026764199618192364, 'depth': 10, 'l2_leaf_reg': 9.853595500270105, 'subsample': 0.13952791716218646, 'random_strength': 0.3063665225874266}. Best is trial 8 with value: 9044.410134286629.


Trial 9 params: {'iterations': 500, 'learning_rate': 0.016584210948173233, 'depth': 10, 'l2_leaf_reg': 8.395189303534835, 'subsample': 0.7418577138356572, 'random_strength': 1.2858201940865714, 'loss_function': 'RMSE', 'eval_metric': 'RMSE', 'random_seed': 42, 'early_stopping_rounds': 50, 'bootstrap_type': 'Bernoulli', 'task_type': 'GPU', 'devices': '0:1:2:3:4:5:6:7:8:9:10'}
0:	learn: 45760.1744630	test: 46033.2737004	best: 46033.2737004 (0)	total: 66.7ms	remaining: 33.3s
10:	learn: 40247.2288453	test: 40475.8659385	best: 40475.8659385 (10)	total: 666ms	remaining: 29.6s
20:	learn: 35652.5896088	test: 35839.0190391	best: 35839.0190391 (20)	total: 1.24s	remaining: 28.2s
30:	learn: 31761.5168520	test: 31927.2379789	best: 31927.2379789 (30)	total: 1.79s	remaining: 27.2s
40:	learn: 28514.1764791	test: 28655.8740623	best: 28655.8740623 (40)	total: 2.36s	remaining: 26.4s
50:	learn: 25767.1498698	test: 25892.7362665	best: 25892.7362665 (50)	total: 2.92s	remaining: 25.7s
60:	learn: 23452.390909

[I 2024-07-18 22:48:12,013] Trial 9 finished with value: 9645.896008681762 and parameters: {'learning_rate': 0.016584210948173233, 'depth': 10, 'l2_leaf_reg': 8.395189303534835, 'subsample': 0.7418577138356572, 'random_strength': 1.2858201940865714}. Best is trial 8 with value: 9044.410134286629.


depth,▁▃██▆▁▁▁██
early_stopping_rounds,▁▁▁▁▁▁▁▁▁▁
iterations,▁▁▁▁▁▁▁▁▁▁
l2_leaf_reg,▃▆▅▇▁▅▄██▆
learning_rate,▂▅▃▂▁▄█▆█▅
random_seed,▁▁▁▁▁▁▁▁▁▁
random_strength,▇▄▃█▄▃▇▁▁▅
rmse,▅▁▂▂█▂▁▂▁▁
subsample,█▅▇▅▅▄▂▇▁▇
bootstrap_type,Bernoulli
depth,10


Best trial:
RMSE: 9044.410134286629
Params: 
    learning_rate: 0.026764199618192364
    depth: 10
    l2_leaf_reg: 9.853595500270105
    subsample: 0.13952791716218646
    random_strength: 0.3063665225874266


In [ ]:
log_data = pd.read_csv(path + 'log_catboost_optuna_colab.csv',index_col=0)
log_data

,iterations,learning_rate,depth,l2_leaf_reg,subsample,random_strength,loss_function,eval_metric,random_seed,early_stopping_rounds,bootstrap_type,task_type,devices,rmse
1,10000,0.058226,9,4.393807,0.693357,1.357619,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:4,5953.608896
2,100000,0.073304,10,4.943193,0.691395,0.995794,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6,5968.725733
3,100000,0.053566,10,4.515512,0.795700,0.986502,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6,5982.645001
0,100000,0.044708,8,5.829044,0.677467,1.798373,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6,5995.046064
0,100000,0.024699,10,3.213412,0.699062,1.642867,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6:7:8:9:10,5998.453308
4,10000,0.054758,9,4.799829,0.717321,0.739163,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:4,6001.441934
2,100000,0.023374,8,4.626836,0.731691,1.699342,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6:7:8:9:10,6003.570028
3,100000,0.016646,8,4.861195,0.889815,0.353491,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6:7:8:9:10,6005.008805
4,100000,0.083325,10,5.517812,0.732981,1.035709,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6,6037.865908
1,100000,0.084768,10,5.703931,0.596415,1.276418,RMSE,RMSE,42,50,Bernoulli,GPU,0:1:2:3:4:5:6,6059.314208


## K-Split Sampling

### lightGBM + TimeSeriesSplit

In [ ]:
# '계약년'과 '계약월'을 문자열로 변환하고 결합하여 날짜 문자열 생성
train_data_encoded['date_str'] = train_data_encoded['계약년'].astype(str) + '-' + train_data_encoded['계약월'].astype(str).str.zfill(2) + '-01'

# 날짜 문자열을 datetime 객체로 변환
train_data_encoded['date'] = pd.to_datetime(train_data_encoded['date_str'])

# 날짜를 기준으로 데이터 정렬
df_sorted = train_data_encoded.sort_values('date')

# 필요 없는 열 삭제
train_data_encoded = df_sorted.drop(['date_str', 'date'], axis=1)

# Target과 독립변수들을 분리
y = train_data_encoded['target']
X = train_data_encoded.drop(['target'], axis=1)

In [ ]:
train_data_encoded.head()

,아파트명,전용면적,층,건축년도,거래유형,k-복도유형,k-난방방식,k-전체세대수,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,좌표X,좌표Y,target,계약월,계약년,구,동,강남여부,distance_score
1118821,2326,52.460,5,1981,0,3,1,122,7354,45,4,127.000071,37.560706,13250,1,2007,23,102,0,5
1101889,6564,84.570,1,1996,0,0,0,278,0,345,0,126.911690,37.495273,34300,1,2007,11,176,0,4
1101924,6253,83.475,7,1988,0,0,0,276,0,345,0,126.968934,37.506779,55000,1,2007,11,335,0,5
1101925,6253,66.225,5,1988,0,0,0,276,0,345,0,126.968934,37.506779,54800,1,2007,11,335,0,5
1101944,539,59.740,6,1999,0,0,0,275,0,345,0,126.950622,37.545915,34000,1,2007,12,13,1,9


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# 파라미터 정의
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'n_estimators': 53149,
    'learning_rate': 0.010204448913134991,
    'num_leaves': 107,
    'max_depth': 10,
    'min_child_samples': 8,
    'subsample': 0.5668227534307944,
    'colsample_bytree': 0.6033862727269486,
    'reg_alpha': 2.5906244698002496e-06,
    'reg_lambda': 2.979573198070953e-05,
}

# TimeSeriesSplit 정의 (예: 5개의 분할)
tscv = TimeSeriesSplit(n_splits=8)

# 모델 학습 및 평가를 위한 리스트
models = []
scores = []


# TimeSeriesSplit을 사용한 교차 검증
for fold, (train_index, val_index) in enumerate(tscv.split(X), 1):
    print(f"Fold {fold}")

    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # LGBMRegressor 모델 생성
    gbm = lgb.LGBMRegressor(**params)

    # 모델 학습
    gbm.fit(X_train_fold, y_train_fold,
            eval_set=[(X_train_fold, y_train_fold), (X_val_fold, y_val_fold)],
            eval_metric='rmse',
            categorical_feature="auto",
            callbacks=[lgb.early_stopping(stopping_rounds=50),
                       lgb.log_evaluation(period=10, show_stdv=True)])

    # 검증 세트에 대한 예측
    y_pred = gbm.predict(X_val_fold)

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
    print(f"Fold {fold} - Validation RMSE: {rmse}")

    models.append(gbm)
    scores.append(rmse)

# 전체 성능 평가
print(f"Average RMSE: {np.mean(scores)}")
print(f"Standard Deviation of RMSE: {np.std(scores)}")

# 가장 좋은 모델 선택 (선택사항)
best_model = models[np.argmin(scores)]

Fold 1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003475 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2155
[LightGBM] [Info] Number of data points in the train set: 124318, number of used features: 18
[LightGBM] [Info] Start training from score 36842.178550
Training until validation scores don't improve for 50 rounds
[10]	training's rmse: 24983.6	valid_1's rmse: 34322.8
[20]	training's rmse: 23286.7	valid_1's rmse: 32109.5
[30]	training's rmse: 21881.1	valid_1's rmse: 30291.4
[40]	training's rmse: 20658.1	valid_1's rmse: 28732.9
[50]	training's rmse: 19320.4	valid_1's rmse: 27014.6
[60]	training's rmse: 18176.5	valid_1's rmse: 25529.2
[70]	training's rmse: 17011.6	valid_1's rmse: 24026.2
[80]	training's rmse: 16137.9	valid_1's rmse: 22875
[90]	training's rmse: 15253.1	valid_1's rmse: 21709.5
[100]	training's rmse: 14452.6	valid_

### lightGBM + optuna + StratifiedKFold

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 데이터 분리
y_train = train_data_encoded['target']
X_train = train_data_encoded.drop(['target'], axis=1)

# 사용할 특성 정의 (모든 열을 사용한다고 가정)
features = X_train.columns.tolist()

# StratifiedKFold 설정
n_folds = 5
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# y_train을 구간으로 나누어 분류 레이블 생성
cut_y_train = pd.cut(y_train, 1000, labels=False)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),
        'verbosity': -1,
        'boosting_type': 'gbdt'
    }

    oof_predictions = np.zeros(len(X_train))

    for fold_idx, (train_idx, valid_idx) in enumerate(kf.split(X_train, cut_y_train)):
        kfold_X_train = X_train.iloc[train_idx][features]
        kfold_y_train = y_train.iloc[train_idx]
        kfold_X_valid = X_train.iloc[valid_idx][features]
        kfold_y_valid = y_train.iloc[valid_idx]

        dtrain = lgb.Dataset(kfold_X_train, label=kfold_y_train)
        dvalid = lgb.Dataset(kfold_X_valid, label=kfold_y_valid, reference=dtrain)

        model = lgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            valid_sets=[dtrain, dvalid],
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
        )

        oof_predictions[valid_idx] = model.predict(kfold_X_valid, num_iteration=model.best_iteration)

    oof_rmse = np.sqrt(mean_squared_error(y_train, oof_predictions))
    return oof_rmse

# Optuna를 사용한 하이퍼파라미터 최적화
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("최적의 하이퍼파라미터:", study.best_params)
print("최적의 RMSE:", study.best_value)

# 최적의 하이퍼파라미터로 최종 모델 학습
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'rmse'

total_oof_predictions = np.zeros(len(X_train))

for fold_idx, (train_idx, valid_idx) in enumerate(kf.split(X_train, cut_y_train)):
    print(f"--------{fold_idx}번째 fold의 학습을 시작합니다.--------")

    kfold_X_train = X_train.iloc[train_idx][features]
    kfold_y_train = y_train.iloc[train_idx]
    kfold_X_valid = X_train.iloc[valid_idx][features]
    kfold_y_valid = y_train.iloc[valid_idx]

    dtrain = lgb.Dataset(kfold_X_train, label=kfold_y_train)
    dvalid = lgb.Dataset(kfold_X_valid, label=kfold_y_valid, reference=dtrain)

    model = lgb.train(
        best_params,
        dtrain,
        num_boost_round=1000,
        valid_sets=[dtrain, dvalid],
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=10)]
    )

    fold_predictions = model.predict(kfold_X_valid, num_iteration=model.best_iteration)
    total_oof_predictions[valid_idx] = fold_predictions

    fold_rmse = np.sqrt(mean_squared_error(kfold_y_valid, fold_predictions))
    print(f"Fold {fold_idx} - Validation RMSE : {fold_rmse}")

final_oof_rmse = np.sqrt(mean_squared_error(y_train, total_oof_predictions))
print(f"최종 OOF RMSE : {final_oof_rmse}")

# 전체 데이터로 최종 모델 학습
dtrain_full = lgb.Dataset(X_train, label=y_train)
final_model = lgb.train(best_params, dtrain_full, num_boost_round=model.best_iteration)

print("모델 학습이 완료되었습니다.")

[I 2024-07-12 00:52:32,307] A new study created in memory with name: no-name-82a9aa81-0926-47e1-b7f2-c13759d03d4e
[I 2024-07-12 00:52:54,225] Trial 0 finished with value: 9701.639746173973 and parameters: {'max_depth': 10, 'learning_rate': 0.10388371092942603, 'subsample': 0.9782791665526014, 'colsample_bytree': 0.5098412502861012, 'min_child_weight': 6, 'lambda_l2': 2.5246446314945077, 'lambda_l1': 0.036998113458092965}. Best is trial 0 with value: 9701.639746173973.
[I 2024-07-12 00:53:11,261] Trial 1 finished with value: 9851.211376154672 and parameters: {'max_depth': 8, 'learning_rate': 0.6022625616695275, 'subsample': 0.6685373918105875, 'colsample_bytree': 0.8135125260460452, 'min_child_weight': 6, 'lambda_l2': 1.643132482159862, 'lambda_l1': 0.0037441516256096078}. Best is trial 0 with value: 9701.639746173973.
[I 2024-07-12 00:53:29,004] Trial 2 finished with value: 9574.847947526852 and parameters: {'max_depth': 7, 'learning_rate': 0.5085491356745381, 'subsample': 0.6925605073

최적의 하이퍼파라미터: {'max_depth': 10, 'learning_rate': 0.26980541055101354, 'subsample': 0.9213002314563011, 'colsample_bytree': 0.5253289763413436, 'min_child_weight': 8, 'lambda_l2': 1.1762326361722986, 'lambda_l1': 0.061708729355894026}
최적의 RMSE: 9069.790746788356
--------0번째 fold의 학습을 시작합니다.--------
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2886
[LightGBM] [Info] Number of data points in the train set: 116298, number of used features: 18
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2

### lightGBM + random seed fold

In [ ]:

# 데이터 분리
y_train = train_data_encoded['target']
X_train = train_data_encoded.drop(['target'], axis=1)

# 사용할 특성 정의 (모든 열을 사용한다고 가정)
features = X_train.columns.tolist()

# 여러 시드 설정
seeds = [i for i in range(10)]  # [0~9]
n_folds = 5

all_oofs = []
all_total_predicts = []
RMSEs = []

for seed in seeds:
    print(f"-------- Seed {seed} --------")

    # StratifiedKFold 설정
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)

    # y_train을 구간으로 나누어 분류 레이블 생성
    cut_y_train = pd.cut(y_train, 1000, labels=False)

    seed_oofs = np.zeros(len(X_train))

    for fold_idx, (train_idx, valid_idx) in enumerate(kf.split(X_train, cut_y_train)):
        print(f"-------- Fold {fold_idx} --------")

        kfold_X_train = X_train.iloc[train_idx][features]
        kfold_y_train = y_train.iloc[train_idx]
        kfold_X_valid = X_train.iloc[valid_idx][features]
        kfold_y_valid = y_train.iloc[valid_idx]

        dtrain = lgb.Dataset(kfold_X_train, label=kfold_y_train)
        dvalid = lgb.Dataset(kfold_X_valid, label=kfold_y_valid, reference=dtrain)

        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'n_estimators': 1000,
            'learning_rate': 0.05,
            'subsample': 0.7,
            'subsample_freq': 1,
            'random_state': seed
        }

        model = lgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            valid_sets=[dtrain, dvalid],
            callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False),
                       lgb.log_evaluation(period=100)]
        )

        seed_oofs[valid_idx] = model.predict(kfold_X_valid, num_iteration=model.best_iteration)

    all_oofs.append(seed_oofs)

    # RMSE 계산
    fold_rmse = np.sqrt(mean_squared_error(y_train, seed_oofs))
    RMSEs.append(fold_rmse)
    print(f"Seed {seed} OOF RMSE: {fold_rmse}")

# 최종 OOF 예측 및 RMSE 계산
final_oof_predictions = np.mean(all_oofs, axis=0)
final_oof_rmse = np.sqrt(mean_squared_error(y_train, final_oof_predictions))
print(f"최종 OOF RMSE: {final_oof_rmse}")

# 전체 데이터로 최종 모델 학습
dtrain_full = lgb.Dataset(X_train, label=y_train)
final_model = lgb.train(params, dtrain_full, num_boost_round=model.best_iteration)
print("모델 학습이 완료되었습니다.")

# 테스트 데이터에 대한 예측 (테스트 데이터가 있다고 가정)
# X_test = test_data[features]
# final_predictions = final_model.predict(X_test, num_iteration=final_model.best_iteration)

-------- Seed 0 --------
-------- Fold 0 --------
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002406 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2887
[LightGBM] [Info] Number of data points in the train set: 124052, number of used features: 18
[LightGBM] [Info] Start training from score 91035.553091
[100]	training's rmse: 18200.9	valid_1's rmse: 18597.9
[200]	training's rmse: 14842.1	valid_1's rmse: 15458.4
[300]	training's rmse: 13176.1	valid_1's rmse: 13920.9
[400]	training's rmse: 12136.8	valid_1's rmse: 12985.3
[500]	training's rmse: 11392.5	valid_1's rmse: 12327.1
[600]	training's rmse: 10825.9	valid_1's rmse: 11851.9
[700]	training's rmse: 10328.6	valid_1's rmse: 11432.6
[800]	training's rmse: 9903.71	valid_1's rmse: 11083
[900]	training's rmse: 9567.46	valid_1's rmse: 10812.9
[1000]	training's rmse: 9271.44	valid_1's rmse

# data ensemble

In [8]:
# catboost의 output 중 validation_rmse가 6054인 데이터와, 
# lgbm 아웃풋 중 public에서 좋은 점수를 받은 output을 합침.
path = './data/'
cat = pd.read_csv(path + 'output_6054.017604271243_6764.193056066714.csv')
lgbm = pd.read_csv(path + 'output_best.csv')

w = [0.97, 0.03]

best = lgbm['target'] * w[0] + cat['target'] * w[1]
submission = pd.DataFrame()
submission['target'] = best.astype(int)

submission.to_csv('submission.csv',index=False)

print("현재 가장 BEST 예측값과 위 앙상블 결과의 rmse : ", np.sqrt(mean_squared_error(lgbm['target'],best)))

현재 가장 BEST 예측값과 위 앙상블 결과의 rmse :  202.92579168200294
